### Dynamic Programming without shortage cost 

#### How to run each test instance


**Instance 1:**  

    1) 
    
    2) Click 'Kernel' in the menu and select 'Restart & Run All' to run the program once.

**Instance 2:** 
    
     1) 
    
     2) Click 'Kernel' in the menu and select 'Restart & Run All' to run the program once.

**Instance 3:** 
    
     1) 
    
     2) Click 'Kernel' in the menu and select 'Restart & Run All' to run the program once.

#### Import Modules

In [ ]:
# reading file
import pandas as pd

#math and random operations
import random as rd
import math
import numpy as np

# Generating graphs
import matplotlib.pyplot as plt; plt.rcdefaults()
import numpy as np
import matplotlib.pyplot as plt

#### Define DP Stages and States

#### Define Variables

#### Define DP Recursion Method

#### Define Objective Value Function

#### Solving DP